In [1]:
import json
import requests
from functools import reduce
import math
import time
from pytz import timezone
import datetime as dt
import schedule
from functools import reduce
import boto3

from resources import ConfigS3

In [2]:
from resources import ConfigS3

In [3]:
define_s3 = ConfigS3()

In [4]:
import pandas as pd

In [4]:
define_s3.upload_file_to_s3("./output.json", "./output_test.json")

In [5]:
API_KEYS_WEATHER = [
  '8df3c71da563b1c6a06e2239c780ba82'
]

LOG_FILE = 'weatherapi_logs.txt'
HCMC = [10.8333, 106.6667]	# https://openweathermap.org/find?q=Ho+Chi+Minh

BASE_URL = f"https://api.openweathermap.org/data/2.5/weather"

In [6]:
def crawl_current_weather():
	params = {
		'lat' : HCMC[0],
		'lon' : HCMC[1],
		'appid' : API_KEYS_WEATHER[0]
	}
	data = requests.get(BASE_URL, params).json()
	result = "none"
	try:
		result = data["weather"][0]["main"]
	except:
		pass
	return result
	

In [7]:
API_KEYS = [
  '21HYqiZMMrc5SrNN6jVOk230BWbg3tAj',
  'Pm7CnuOA6G6Uw25xaMtGy6yxSmaqACXM',
  'cBAeCxqGO4OJq7y1wfyFOhA0HRWpgAml',
  'nGKarNyeU0VtPjtM5chG4Uif6KlRUa1E',
  '1j5sGYeflskicFlJYJtMbgFdLB9xRAji'
]

In [8]:
api_counters = [0] * len(API_KEYS)
api_call_counter = 0
retry = 0
MAX_RETRY = 10
LOG_FILE = 'tomtom_logs.txt'

In [9]:
checked_time_list = {
   "1": ["21:55:26", "22:55:26"],
   "2": ["8:00:00", "10:00:00"]
}

In [10]:
STYLES = ['absolute', 'relative', 'relative0', 'relate0-dark', 'relative-delay', 'reduced-sensitivity']
ZOOM = 22
TOMTOM_URL = f"https://api.tomtom.com/traffic/services/4/flowSegmentData/{STYLES[0]}/{ZOOM}/json"



In [11]:
define_s3 = ConfigS3()

In [12]:
def convert_str_to_time(time_str):
   return dt.datetime.strptime(time_str, '%H:%M:%S').time()

In [13]:
def check_times(list_time):
   tz_VN = timezone('Asia/Ho_Chi_Minh') 
   datetime_VN = dt.datetime.now(tz_VN)
   current_time = datetime_VN.time()
   checked_time = False
   for key, value in checked_time_list.items():
      print("current time:", current_time)
      print(value)
      if convert_str_to_time(value[0]) <= current_time <= convert_str_to_time(value[1]):
        checked_time = True

   return checked_time

In [14]:
def log(message):
  with open(LOG_FILE, 'a+') as f:
    f.write(message)

In [15]:
def tom_url(zoom_level): return f"https://api.tomtom.com/traffic/services/4/flowSegmentData/{STYLES[0]}/{zoom_level}/json"


In [16]:
def get_tomtom_data(lat, lng, zoom_level=22):
    global api_counters
    global api_call_counter

    try:
        params = {
          'point': f"{lat},{lng}",
          'unit': 'KMPH',
          'openLr': 'false',
          'key': API_KEYS[api_call_counter]
        }
        data = requests.get(tom_url(zoom_level), params=params).json()

        api_counters[api_call_counter] += 1
        return data['flowSegmentData']
        
    except ValueError:
        api_call_counter += 1
        return get_tomtom_data(lat, lng)
        
    except KeyError:
        log("Key error: " + str(lat) + "," + str(lng) + "\n")
        return None

In [18]:
data = get_tomtom_data(10.67401655950468, 106.58051223264462)
print(data['currentSpeed'])

52


In [70]:
def crawl_data(limit=40):
    """
    Main function to crawl data an
    d map to segment_id
    """
    global checked_time_list
    if not check_times(checked_time_list):
       print("000000000")
       return
    with open('selected_points.json', 'r') as f:
        cover_points = json.load(f)
    weather = crawl_current_weather()
    timestamp = int(time.time())
    output = dict()
    id = 0
    for record in cover_points:
        print(record)
        for point in record["segment_ids"]:
            lng, lat = point['lng'], point['lat']
            tomtom_data = get_tomtom_data(lat, lng)
            if tomtom_data == None: continue
            temp_out = {
                'timestamp': timestamp,
                'velocity': tomtom_data['currentSpeed'],
                'confidence': tomtom_data['confidence'],
                'segment_id': point['segment_id'],
                'source': 'tom-tom',
                'weather': weather
            }
            print(temp_out)
            output[point["segment_id"]] = temp_out
        id += 1
        print(f"------{id}------")
        if id == limit: break

    with open('output.json', 'w') as f:
        json.dump(output, f)

    # define_s3.upload_file_to_s3('output.json', f'tomtom-voh/{timestamp}.json')
    # s3 = boto3.client('s3')
    # with open('output.json', 'rb') as f:
    #     s3.upload_fileobj(f, 'tomtom-voh', f"{timestamp}.json")
                
    
    return timestamp

In [71]:
crawl_data()

current time: 22:02:05.045380
['21:55:26', '22:55:26']
current time: 22:02:05.045380
['8:00:00', '10:00:00']
{'name': 'GO VAP : NGUYEN OANH - PHAN VAN TRI', 'point': [{'lat': 10.830940288506234, 'lng': 106.67745796558785}], 'segment_ids': [{'lat': 10.8309854, 'lng': 106.6773216, 'segment_id': 23795}, {'lat': 10.831059100000001, 'lng': 106.67738605, 'segment_id': 60703}, {'lat': 10.83091555, 'lng': 106.67752045, 'segment_id': 23794}, {'lat': 10.83089375, 'lng': 106.6774991, 'segment_id': 60704}, {'lat': 10.8309244, 'lng': 106.6773346, 'segment_id': 60538}, {'lat': 10.83105175, 'lng': 106.67746894999999, 'segment_id': 65249}, {'lat': 10.830838700000001, 'lng': 106.67757019999999, 'segment_id': 60539}, {'lat': 10.83077275, 'lng': 106.67758309999999, 'segment_id': 60705}, {'lat': 10.83087855, 'lng': 106.6776209, 'segment_id': 23793}, {'lat': 10.8308605, 'lng': 106.67759154999999, 'segment_id': 65248}], 'district': 'quan_go_vap'}
{'timestamp': 1680188525, 'velocity': 36, 'confidence': 1, 's

1680188525